# Online

In [ ]:
#! pip download -d wheels torch==1.7.1 torchvision==0.8.2 pytorch-lightning==1.2.6 pytorch-metric-learning==0.9.98 faiss-gpu==1.7.0 pydantic==1.7 imbalanced-learn==0.7 aurum==0.3.1 stripping==0.2.6 minio==6.0 albumentations==0.5.2 scikit-image==0.16.2

In [ ]:
#! rm wheels/PyYAML-*
#! rm wheels/urllib3-*
#! rm wheels/tensorboard-*
#! rm wheels/numpy-*
#! rm wheels/Pillow-*

In [ ]:
#import os
#from glob import glob
#for tar_file in glob("wheels/*.tar.gz"):
#    os.rename(tar_file, tar_file.replace(".tar.gz", ".xyz"))

In [ ]:
#! zip -r wheels.zip wheels/

In [ ]:
#from IPython.display import FileLink
#FileLink(r'wheels.zip')

# Offline

In [ ]:
! mkdir -p /tmp/wheels

In [ ]:
! cp /kaggle/input/shopeewheels/wheels/* /tmp/wheels

In [ ]:
import os
from glob import glob

targzs = glob("/tmp/wheels/*.xyz")
for targz in targzs:
    print(targz)
    if "timm" in targz:
        print(targz)
        os.remove(targz)
    else:
        os.rename(targz, targz.replace(".xyz", ".tar.gz"))

In [ ]:
targzs = glob("/tmp/wheels/*")
for targz in targzs:
     if "timm" in targz:
        print(targz)
        os.remove(targz)

In [ ]:
! pip install --no-index --no-deps /tmp/wheels/*

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
import timm
timm.__version__

In [ ]:
experiment_ids = ["b89623cd-164a-4ce9-9458-96a2a7df21a1", "f04628b3-85b4-4182-8110-8f08a07b188a", "e4cae871-accf-4ed1-b316-7dcc2aae9883", "c0eb698d-adb4-46b9-a841-f6f4157725e1"]
experiment_id_no_slashes = experiment_ids[-1].replace("-", "")
distance_thresholds = [[0.8]] * 4
#distance_threshold_regressor_paths = ["39261eb9d94646a8b5ba649eddb29c73-mlp-thr-reg/39261eb9-d946-46a8-b5ba-649eddb29c73_mlp_thr_reg.pt"]
distance_threshold_regressor_paths = [None] * 4
ensemble_method = "AFFIRMATIVE"
apply_lower_case=False
apply_clean_numbers=False
apply_clean_characters=False
apply_normalize_title=False
apply_remove_stopwords=False
text_embedding_dim=25000
tf_idf_embedding_strategy = "part_of_ensemble" # ["union_after_ensemble", "part_of_ensemble"]

In [ ]:
! git config --global user.email "fernando@xnv.io"
! git config --global user.name "Fernando Camargo"

In [ ]:
! git init
! au init

In [ ]:
! cp -r /kaggle/input/$experiment_id_no_slashes/* .

In [ ]:
try:
    import aurum as au
except:
    pass # It's raising an exception the first time for some reason

In [ ]:
from typing import *
import pickle
from multiprocessing import Pool
import functools

import numpy as np
import pandas as pd
import torch
from sklearn.base import RegressorMixin
from torch.utils.data.dataset import Dataset
import PIL.Image
from tqdm.notebook import tqdm
import albumentations as A
from albumentations import Compose
import faiss

from shopee.model.generic_triplet import *
from shopee.model.generic_metric_learner import *
from shopee.data_preparation import ImagePreprocessor, TextPreprocessor
from shopee.dataset import ShopeeDataset
from shopee.ensembling import ensemble, EnsembleMethod
from shopee.nlp.sequence import pad_sequences
#from shopee.torch.utils import transfer_batch_to_device

In [ ]:
import numpy as np 
import pandas as pd 

import math
import random 
import os 
import cv2
import timm

from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset 
from torch import nn
import torch.nn.functional as F 

import gc
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import re
import unicodedata
import os

In [ ]:
import PIL.Image

from shopee.data_preparation import PIL_INTERPOLATION_METHODS
from shopee.dataset import NoOpImagePreprocessor
from shopee.utils import (
    apply_clahe,
    resize_keeping_aspect_ratio,
)

def transfer_batch_to_device(
    batch: Union[torch.Tensor, list, dict], device: torch.device
) -> Union[torch.Tensor, list, dict]:
    # base case
    if isinstance(batch, torch.Tensor):
        return batch.to(device)

    # when list/tuple
    elif isinstance(batch, list) or isinstance(batch, tuple):
        for i, x in enumerate(batch):
            batch[i] = transfer_batch_to_device(x, device)
        return batch

    # when dict
    elif isinstance(batch, dict):
        for k, v in batch.items():
            batch[k] = transfer_batch_to_device(v, device)

        return batch

    # nothing matches, return the value as is without transform
    return batch

class ImagePreprocessor(object):
    def __init__(
        self,
        image_size: Tuple[int, int],
        keep_aspect_ratio: bool,
        apply_clahe: bool,
        interpolation_method: int,
        center_crop: bool,
    ) -> None:
        super().__init__()
        self._image_size = image_size
        self._keep_aspect_ratio = keep_aspect_ratio
        self._apply_clahe = apply_clahe
        self._interpolation_method = interpolation_method
        self._center_crop = center_crop

    @property
    def suffix(self) -> str:
        suffix = "%d_%d" % self._image_size
        if self._keep_aspect_ratio:
            suffix += "_kept_aspect_ratio"
        if self._apply_clahe:
            suffix += "_clahe"
        if self._center_crop:
            suffix += "_cropped"
        suffix += f"_interpolation={self._interpolation_method}"

        return suffix

    def preprocess_image(self, image: PIL.Image.Image) -> PIL.Image.Image:
        if self._keep_aspect_ratio:
            image = resize_keeping_aspect_ratio(
                image, self._image_size, self._interpolation_method
            )
        elif self._center_crop:
            width, height = image.size
            if width < self._image_size[0] or height < self._image_size[1]:
                image = image.resize(self._image_size, self._interpolation_method)
            image = np.array(image)
            transform = A.CenterCrop(
                height=self._image_size[0], width=self._image_size[1]
            )
            image = PIL.Image.fromarray(transform(image=image)["image"])

        else:
            image = image.resize(self._image_size, self._interpolation_method)
        if self._apply_clahe:
            image = apply_clahe(image)
        return image

    def preprocess_file(self, image_filepath: str, output_dir: str) -> PIL.Image.Image:
        image = PIL.Image.open(image_filepath)
        image = self.preprocess_image(image)
        image.save(
            os.path.join(output_dir, os.path.split(image_filepath)[1]), quality=100
        )

    def preprocess_dir(self, input_dir: str, output_dir: str):
        image_paths = glob(os.path.join(input_dir, "*.jpg"))
        with Pool(os.cpu_count()) as pool:
            list(
                tqdm(
                    pool.imap(
                        functools.partial(self.preprocess_file, output_dir=output_dir),
                        image_paths,
                    ),
                    total=len(image_paths),
                )
            )
            

class ShopeeDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        images_dir: str,
        image_preprocessing_function: Callable[[np.ndarray], np.ndarray],
        image_preprocessor: ImagePreprocessor = NoOpImagePreprocessor(),
        original_images_dir: str = None,
        text_embeddings: np.ndarray = None,
        augmenter: A.Compose = None,
        auto_augment: str = None,
        test: bool = False,
    ) -> None:
        super().__init__()

        self._df = df
        self._images_dir = images_dir
        self._image_preprocessing_function = image_preprocessing_function
        self._image_preprocessor = image_preprocessor
        self._original_images_dir = original_images_dir
        self._text_embeddings = text_embeddings
        self._augmenter = augmenter
        self._auto_augment = (
            auto_augment_transform(auto_augment, None) if auto_augment else None
        )
        self._test = test

    def __len__(self) -> int:
        return len(self._df)

    def get_label_groups(self) -> np.ndarray:
        return self._df["label_group"].values

    def _get_preprocessed_image(self, image_filename: str) -> np.ndarray:
        image_path = os.path.join(self._images_dir, image_filename)

        if not os.path.exists(image_path):
            original_image_path = os.path.join(self._original_images_dir, image_filename)
            image = self._image_preprocessor.preprocess_file(original_image_path, self._images_dir)
            
        image = PIL.Image.open(image_path)
        
        if self._auto_augment is not None:
            image = self._auto_augment(image)
        return np.array(image)

    def __getitem__(
        self, index: int
    ) -> Union[
        Tuple[Dict[str, torch.Tensor], torch.Tensor, torch.Tensor],
        Dict[str, torch.Tensor],
    ]:
        item = self._df.iloc[index]

        image = self._get_preprocessed_image(item["image"])

        if self._augmenter is not None:
            image = self._augmenter(image=image)["image"]

        image = self._image_preprocessing_function(image)
        image = image.transpose(2, 0, 1)

        image_tensor = torch.tensor(image, dtype=torch.float32)

        inputs = {"images": image_tensor}

        if self._text_embeddings is not None:
            text_tensor = torch.tensor(self._text_embeddings[index], dtype=torch.float32)
            inputs['texts'] = text_tensor

        if self._test:
            return inputs
        else:
            return (
                inputs,
                torch.tensor(item["label_group"], dtype=torch.long),
                torch.tensor(item["sample_weight"], dtype=torch.float32)
                if "sample_weight" in item
                else torch.tensor(1.0, dtype=torch.float32),
            )

In [ ]:
from IPython.display import display, clear_output

In [ ]:
def get_images_dir(image_preprocessor: ImagePreprocessor) -> str:
    return os.path.join(f"/tmp/images_{image_preprocessor.suffix}")

In [ ]:
from torch.utils.data.dataloader import DataLoader
from pytorch_metric_learning.distances import (
    CosineSimilarity,
    DotProductSimilarity,
    LpDistance,
    SNRDistance,
)

def _predict_matches(
    self, loader: DataLoader, device: torch.device, index: faiss.Index, is_inverted: bool, k: int = 50, distance_threshold: float = np.inf, distance_threshold_regressor: RegressorMixin = None,
) -> List[np.ndarray]:
    self.eval()

    embeddings_list: List[torch.Tensor] = []
    if distance_threshold_regressor is not None:
        distance_thresholds_list = []
    with torch.no_grad():
        for inputs in tqdm(loader, total=len(loader)):
            emb = self(transfer_batch_to_device(inputs, device))
            #if isinstance(self.distance, CosineSimilarity):
            emb = F.normalize(emb)
            embeddings = emb.cpu().numpy()
            embeddings_list.append(embeddings)
            if distance_threshold_regressor is not None:
                if isinstance(distance_threshold_regressor, torch.jit.ScriptModule):
                    distance_thresholds_list.append(distance_threshold_regressor(emb).flatten().cpu().numpy())
                else:
                    distance_thresholds_list.append(distance_threshold_regressor.predict(embeddings))
            
    self.to(torch.device("cpu"))
    torch.cuda.empty_cache()

    embeddings = np.concatenate(embeddings_list, axis=0)
    
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
    index.add(embeddings)
    all_distances, all_indices = index.search(embeddings, k)

    def match(distance: float, distance_threshold: float) -> bool:
        return (
            distance >= distance_threshold
            if is_inverted
            else distance <= distance_threshold
        )
    
    if distance_threshold_regressor is not None:
        distance_thresholds = np.concatenate(distance_thresholds_list, axis=0)
        return [
            np.array([i for distance, i in zip(distances, indices) if match(distance, min(distance_threshold + 0.1, 1.0))])
            for distances, indices, distance_threshold in zip(all_distances, all_indices, distance_thresholds)
        ]
    else:
        return [
            np.array([i for distance, i in zip(distances, indices) if match(distance, distance_threshold)])
            for distances, indices in zip(all_distances, all_indices)
        ]

In [ ]:
def predict_matches(experiment_id: str, test_df: pd.DataFrame, input_images_dir: str, distance_thresholds: List[float], distance_threshold_regressor_path: str = None, tta_steps: int = 0, device: torch.device = torch.device("cpu"), augmenter: Compose = None) -> List[np.ndarray]:
    display(f"Predicting using {experiment_id}...")
    
    if distance_threshold_regressor_path:
        try:
            distance_threshold_regressor = torch.jit.load(os.path.join("/kaggle/input", distance_threshold_regressor_path)).to(device)
        except:
            with open(os.path.join("/kaggle/input", distance_threshold_regressor_path), "rb") as f:
                distance_threshold_regressor = pickle.load(f)
    else:
        distance_threshold_regressor = None
    
    base_model_path = f"/kaggle/input/{experiment_id.replace('-', '')}/output/{experiment_id}/models"

    posting_ids = test_df["posting_id"].values
    
    folds_outputs = []

    for n_fold, distance_threshold in zip(sorted(os.listdir(base_model_path)), distance_thresholds):
        if not os.path.isdir(os.path.join(base_model_path, n_fold)):
            continue
            
        df = test_df.copy()
        
        display(f"Loading model for {n_fold}...")
        model_path = os.path.join(base_model_path, n_fold, "model.pt")
        script_model_path = os.path.join(base_model_path, n_fold, "script_model.pt")
        if os.path.exists(script_model_path):
            display(f"Found TorchScript model for {n_fold}")
            model = torch.load(model_path, map_location=torch.device("cpu")).to(torch.device("cpu"))
            script_model = torch.jit.load(script_model_path, map_location=device).to(device)
        else:
            display(f"TorchScript model for {n_fold} not found")
            model = torch.load(model_path, map_location=device).to(device)
            script_model = None
            
        with open(os.path.join(base_model_path, n_fold, "image_preprocessor.pkl"), "rb") as f:
            image_preprocessor = pickle.load(f)
            
        if os.path.exists(os.path.join(base_model_path, n_fold, "text_preprocessor.pkl")):
            with open(os.path.join(base_model_path, n_fold, "text_preprocessor.pkl"), "rb") as f:
                text_preprocessor = pickle.load(f)
                text_preprocessor.preprocess_text(df)
                text_embeddings = text_preprocessor.transform(df)
                text_embeddings = pad_sequences(text_embeddings, maxlen=15)
        else:
            text_embeddings = None
            
        if augmenter is None:
            with open(os.path.join(base_model_path, n_fold, "augmenter.pkl"), "rb") as f:
                augmenter = pickle.load(f)
                
        with open(os.path.join(base_model_path, n_fold, "image_preprocessing_function.pkl"), "rb") as f:
            image_preprocessing_function = pickle.load(f)
        
        images_dir = get_images_dir(image_preprocessor)
        if not os.path.exists(images_dir):
            #display("Preprocessing images...")
            os.makedirs(images_dir)
            #image_preprocessor.preprocess_dir(input_images_dir, images_dir)
            #display("Images preprocessed successfully.")
        
        dataset = ShopeeDataset(
            df,
            images_dir,
            image_preprocessing_function,
            image_preprocessor=image_preprocessor,
            original_images_dir=input_images_dir,
            text_embeddings=text_embeddings,
            #image_preprocessor=image_preprocessor,
            augmenter=augmenter if tta_steps else None,
            test=True,
        )
        
        loader = DataLoader(
            dataset,
            batch_size=model.training_config.batch_size * 2,
            shuffle=False,
            pin_memory=True,
            num_workers=os.cpu_count(),
            prefetch_factor=10,
        )

        display("Running the model...")
        index = model._faiss_index_type(model.model_config.embedding_dim if getattr(model.model_config, "use_output_fc", True)  else model.get_embedding_dim())
        match_indices = _predict_matches(
            script_model if script_model is not None else model,
            loader,
            device,
            index,
            model.distance.is_inverted,
            distance_threshold=distance_threshold,
            distance_threshold_regressor=distance_threshold_regressor
        )
        #match_indices = model.predict_matches(dataset, distance_threshold=distance_threshold)
        matches = [posting_ids[indices].tolist() for indices in match_indices]
        
        folds_outputs.append(matches)
        #folds_outputs.append(predict_proba(model, dataset, tta_steps=tta_steps))
        display("Model ran successfully.")
        
    return folds_outputs

In [ ]:
df = pd.read_csv("/kaggle/input/shopee-product-matching/test.csv")
input_images_dir = "/kaggle/input/shopee-product-matching/test_images"

In [ ]:
#df = pd.read_csv("/kaggle/input/shopee-product-matching/train.csv")
#input_images_dir = "/kaggle/input/shopee-product-matching/train_images"

In [ ]:
experiment_tta_matches = [
    predict_matches(experiment_id, df, input_images_dir, distance_thresholds=exp_distance_thresholds, distance_threshold_regressor_path=distance_threshold_regressor_path, device=torch.device("cuda"))
    for experiment_id, exp_distance_thresholds, distance_threshold_regressor_path in zip(experiment_ids, distance_thresholds, distance_threshold_regressor_paths)
]

In [ ]:
all_matches = [matches for tta_matches in experiment_tta_matches for matches in tta_matches]

In [ ]:
def read_dataset():
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']
    return df_cu, image_paths

In [ ]:
cachedStopWords = stopwords.words("indonesian")
def clean_numbers(x):

    x = re.sub('[0-9]', '0', x)
    return x


def clean_characters(x):
    print("Cleaning")

    x = str(x)
    for punct in "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&':
        x = x.replace(punct, ' ')
    return x

def normalize_title(title):
    return unicodedata.normalize('NFKD', title.lower()).encode('ASCII', 'ignore').decode('utf8')


def remove_stopwords(title):

    return ' '.join([word for word in title.split() if word not in cachedStopWords])

In [ ]:
def preprocess_text(dataframe):
    if apply_lower_case:
        print("Applying lower case")
        dataframe["title"] = dataframe["title"].str.lower()
    
    if apply_clean_numbers:
        print("Applying clean numbers")
        dataframe["title"] = dataframe["title"].apply(
            lambda x: clean_numbers(x)
        )

    if apply_clean_characters:
        print("Applying clean characters")
        dataframe["title"] = dataframe["title"].apply(
            lambda x: clean_characters(x)
        )

    if apply_normalize_title:
        print("Applying normalize title")
        dataframe["title"] = dataframe["title"].apply(
            lambda x: normalize_title(x)
        )
        
    if apply_remove_stopwords:
        print("Removing stop words")
        dataframe["title"] = dataframe["title"].apply(
            lambda x: remove_stopwords(x)
        )
    return dataframe

In [ ]:
df = preprocess_text(df)

In [ ]:
df_cu,image_paths = read_dataset()

In [ ]:
def get_text_predictions(df, max_features = 10000):
    
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.75)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
    
    del model,text_embeddings
    gc.collect()
    return preds

In [ ]:
tf_idf_matches = get_text_predictions(df, max_features = text_embedding_dim)

In [ ]:
import math
from collections import Counter
from typing import List, Union

def _intersection(*matches: List[Union[int, str]]) -> List[Union[int, str]]:
    sets = iter(map(set, matches))
    result = next(sets)
    for s in sets:
        result = result.intersection(s)
    return list(result)


def _union(*matches: List[Union[int, str]]) -> List[Union[int, str]]:
    sets = iter(map(set, matches))
    result = next(sets)
    for s in sets:
        result = result.union(s)
    return list(result)


def ensemble(
    model_matches: List[List[Union[int, str]]],
    num_models: int = None,
    ensemble_method: EnsembleMethod = EnsembleMethod.CONSENSUS,
) -> List[Union[int, str]]:
    if num_models is None:
        num_models = len(model_matches)

    if ensemble_method == EnsembleMethod.CONSENSUS:
        counter = Counter([match for matches in model_matches for match in matches])
        cutoff = math.ceil(num_models / 2)
        return [match for match, count in counter.items() if count >= cutoff]
    elif ensemble_method == EnsembleMethod.UNANIMOUS:
        return _intersection(*model_matches)
    else:
        return _union(*model_matches)

In [ ]:
if tf_idf_embedding_strategy == "union_after_ensemble": # ["union_after_ensemble", "part_of_ensemble"]
    with Pool(os.cpu_count()) as pool:
        ensembled_matches = pool.map(functools.partial(ensemble, ensemble_method=EnsembleMethod[ensemble_method]), list(zip(*all_matches)))
        ensembled_matches = pool.map(functools.partial(ensemble, ensemble_method=EnsembleMethod.AFFIRMATIVE), list(zip(ensembled_matches, tf_idf_matches)))
else:
    all_matches.append(tf_idf_matches)
    with Pool(os.cpu_count()) as pool:
        ensembled_matches = pool.map(functools.partial(ensemble, ensemble_method=EnsembleMethod[ensemble_method]), list(zip(*all_matches)))

In [ ]:
submission_df = df[["posting_id"]]
submission_df["matches"] = [" ".join(matches) for matches in ensembled_matches]

In [ ]:
submission_df.to_csv("submission.csv", index=False)

In [ ]:
submission_df